In [15]:
import requests
import json
headers = {"accept": "application/json"}

In [16]:
#read the json file and grab the api-root and api-key
with open('api_config.json') as f:
    data = json.load(f)
    api_root = data['api-root']
    api_key = data['api-key']

In [31]:
#first grabbing the league heirarchy to get our two teams of interest (Lakers and Timberwolves)
url = api_root + '/league/hierarchy.json?api_key=' + api_key
print(url)
response = requests.get(url, headers=headers)
if response.status_code == 200:
    print("Success")
    league_string = response.text
    league_heirarchy = json.loads(league_string)
#save json to file
with open('league_heirarchy.json', 'w') as f:
    json.dump(league_heirarchy, f, indent=4)

https://api.sportradar.us/nba/trial/v8/en/league/hierarchy.json?api_key=ojfyf7N58ypD8zvafp4Ce3BdShCHLGtDGTyY74dz
Success


In [44]:
#for the ease of use, lets convert our league heigharchy to a dataframe where each row is a team
import pandas as pd
#the headers of the dataframe are as follows, conference, division, teamName, teamId, teamCity
df = pd.DataFrame(columns=['conference', 'division', 'teamName', 'teamId', 'teamCity'])
for conference in league_heirarchy['conferences']:
    for division in conference['divisions']:
        for team in division['teams']:
            df = df.append({'conference': conference['name'], 'division': division['name'], 'teamName': team['name'], 'teamId': team['id'], 'teamCity': team['market']}, ignore_index=True)
#save df to csv
df.to_csv('league_heirarchy.csv', index=False)

C:\Users\pnihr\AppData\Local\Temp\ipykernel_40724\4057848420.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'conference': conference['name'], 'division': division['name'], 'teamName': team['name'], 'teamId': team['id'], 'teamCity': team['market']}, ignore_index=True)
C:\Users\pnihr\AppData\Local\Temp\ipykernel_40724\4057848420.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'conference': conference['name'], 'division': division['name'], 'teamName': team['name'], 'teamId': team['id'], 'teamCity': team['market']}, ignore_index=True)
C:\Users\pnihr\AppData\Local\Temp\ipykernel_40724\4057848420.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'conference': conference['

In [50]:
#Inside of the dataframe, we can now filter for the Lakers and Timberwolves and grab their teamId
lakers_id = df[df['teamName'] == 'Lakers']['teamId'].item()
timberwolves_id = df[df['teamName'] == 'Timberwolves']['teamId'].item()
print("Lakers ID: " + lakers_id)
print("Timberwolves ID: " + timberwolves_id)

Lakers ID: 583ecae2-fb46-11e1-82cb-f4ce4684ea4c
Timberwolves ID: 583eca2f-fb46-11e1-82cb-f4ce4684ea4c


In [66]:
def normalize_json(data: dict) -> dict: 
  
    new_data = dict() 
    for key, value in data.items(): 
        if not isinstance(value, dict): 
            new_data[key] = value 
        else: 
            for k, v in value.items(): 
                new_data[key + "_" + k] = v 
      
    return new_data 

In [56]:
#Now that we have the team ids, we can grab the team stats
#Notice I am only grabbing from 2018 onwards for the statistics
#I believe this may project better as we are only considering the "Lebron James era" in LA
#2024 season has not started yet, so no need to grab that data
lakers_df = pd.DataFrame(columns=['year', 'id', 'name', 'fgm', 'fga', 'fg%', '2pm', '2pa', '2p%', '3pm', '3pa', '3p%', 'blockeda', 'ftm', 'fta', 'ft%', 'oreb', 'dreb', 'reb', 'ast', 'tov', 'stl', 'blk', 'fouls', 'pts', 'fbpts', 'ptsoffto', '2ndpts', 'ptsinpaint', 'ptsinpainta', 'ptsinpaintm', 'effiency', 'true_shooting_att','bench_pts', 'fouls_drawn','o_fouls','o_rating','d_rating', 'gamesPlayed'
                                  'opp_fgm', 'opp_fga', 'opp_fg%', 'opp_2pm', 'opp_2pa', 'opp_2p%', 'opp_3pm', 'opp_3pa', 'opp_3p%', 'opp_blockeda', 'opp_ftm', 'opp_fta', 'opp_ft%', 'opp_oreb', 'opp_dreb', 'opp_reb', 'opp_ast', 'opp_tov', 'opp_stl', 'opp_blk', 'opp_fouls', 'opp_pts', 'opp_fbpts', 'opp_ptsoffto', 'opp_2ndpts', 'opp_ptsinpaint', 'opp_ptsinpainta', 'opp_ptsinpaintm', 'opp_effiency', 'opp_true_shooting_att','opp_bench_pts', 'opp_fouls_drawn','opp_o_fouls','opp_o_rating','opp_d_rating', 'opp_gamesPlayed'])
timberwolves_df = pd.DataFrame(columns=['year', 'teamId', 'teamName', 'fgm', 'fga', 'fg%', '2pm', '2pa', '2p%', '3pm', '3pa', '3p%', 'blockeda', 'ftm', 'fta', 'ft%', 'oreb', 'dreb', 'reb', 'ast', 'tov', 'stl', 'blk', 'fouls', 'pts', 'fbpts', 'ptsoffto', '2ndpts', 'ptsinpaint', 'ptsinpainta', 'ptsinpaintm', 'effiency', 'true_shooting_att','bench_pts', 'fouls_drawn','o_fouls','o_rating','d_rating', 'gamesPlayed'
                                  'opp_fgm', 'opp_fga', 'opp_fg%', 'opp_2pm', 'opp_2pa', 'opp_2p%', 'opp_3pm', 'opp_3pa', 'opp_3p%', 'opp_blockeda', 'opp_ftm', 'opp_fta', 'opp_ft%', 'opp_oreb', 'opp_dreb', 'opp_reb', 'opp_ast', 'opp_tov', 'opp_stl', 'opp_blk', 'opp_fouls', 'opp_pts', 'opp_fbpts', 'opp_ptsoffto', 'opp_2ndpts', 'opp_ptsinpaint', 'opp_ptsinpainta', 'opp_ptsinpaintm', 'opp_effiency', 'opp_true_shooting_att','opp_bench_pts', 'opp_fouls_drawn','opp_o_fouls','opp_o_rating','opp_d_rating', 'opp_gamesPlayed'])
for year in range(2019, 2024):
    lakers_url = api_root + '/seasons/{year}/REG/teams/{team_id}/statistics.json?api_key={api_key}'.format(year=year, team_id=lakers_id, api_key=api_key)
    timberwolves_url = api_root + '/seasons/{year}/REG/teams/{team_id}/statistics.json?api_key={api_key}'.format(year=year, team_id=lakers_id, api_key=api_key)
    lakers_response = requests.get(lakers_url, headers=headers)
    timberwolves_response = requests.get(timberwolves_url, headers=headers)
    if lakers_response.status_code == 200:
        print("Success")
        lakers_string = lakers_response.text
        lakers_stats = json.loads(lakers_string)
    if timberwolves_response.status_code == 200:
        print("Success")
        timberwolves_string = timberwolves_response.text
        timberwolves_stats = json.loads(timberwolves_string)
    

                                  

Success
Success
Success
Success


In [86]:
nba_team_stats = pd.json_normalize(lakers_stats)
nba_players = pd.DataFrame()
players_list = nba_team_stats['players']
for player in players_list:
    player_df = pd.json_normalize(player)
    nba_players = nba_players.append(player_df)


C:\Users\pnihr\AppData\Local\Temp\ipykernel_40724\1134795131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lakers_players = lakers_players.append(player_df)


In [85]:
lakers_json = normalize_json(lakers_stats)
lakers_json = normalize_json(lakers_json)
print(lakers_json)
#turn the json into a dataframe
lakers_df = pd.DataFrame(lakers_json)
print(lakers_df)
lakers_df.drop(columns=['players'], inplace=True)
lakers_df.to_csv('lakers_stats.csv', index=False)

{'id': '583ecae2-fb46-11e1-82cb-f4ce4684ea4c', 'name': 'Lakers', 'market': 'Los Angeles', 'sr_id': 'sr:team:3427', 'reference': '1610612747', 'season_id': '5027b6ac-731c-4622-8d69-d863ae7c626b', 'season_year': 2022, 'season_type': 'REG', 'own_record_total_games_played': 82, 'own_record_total_minutes': 19880.0, 'own_record_total_field_goals_made': 3516, 'own_record_total_field_goals_att': 7298, 'own_record_total_field_goals_pct': 0.482, 'own_record_total_two_points_made': 2631, 'own_record_total_two_points_att': 4740, 'own_record_total_two_points_pct': 0.555, 'own_record_total_three_points_made': 885, 'own_record_total_three_points_att': 2558, 'own_record_total_three_points_pct': 0.346, 'own_record_total_blocked_att': 416, 'own_record_total_free_throws_made': 1691, 'own_record_total_free_throws_att': 2182, 'own_record_total_free_throws_pct': 0.775, 'own_record_total_offensive_rebounds': 818, 'own_record_total_defensive_rebounds': 2930, 'own_record_total_rebounds': 3748, 'own_record_tota